<a href="https://colab.research.google.com/github/deacs11/CrewAI_Market_Analysis/blob/main/CrewAI_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Necessary Libraries
# This command installs crewai, its basic tools, the OpenAI client,
# the Colab interaction module, and the DuckDuckGo search tool.
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q
!pip install --upgrade crewai

print("Library installation completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta

In [2]:
# @title 2. Import Modules and Configure API Keys from Secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool # Useful tools
from langchain_openai import ChatOpenAI

# --- API KEY CONFIGURATION FROM COLAB SECRETS ---
print("Attempting to load API Keys from Secrets...")
try:
    # Reads the OpenAI key from the secret named 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret named 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API Key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API Keys from Secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify the OpenAI key in the environment
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    # Show only start and end for security
    print(f"OpenAI Key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI Key NOT FOUND in environment.")
    print("!!! Check Colab Secrets and re-run this cell.")
print("-" * 20)
# Also check the Serper key if present
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper Key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper Key NOT FOUND (required for SerperDevTool).")
print("-" * 20)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

Attempting to load API Keys from Secrets...
-> API Key environment variables set successfully.
--------------------
OpenAI Key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper Key FOUND in environment: '7354c...dc56'
--------------------


In [3]:
# @title 3. Define the Sector/Market to Analyze

# --- MODIFY THIS STRING WITH THE SECTOR YOU ARE INTERESTED IN ---
# Examples: "Global Electric Vehicle Market", "Professional Video Editing Software",
#         "Cloud Computing Trends in Europe", "Major Fast Food Chains in the USA"
sector_to_analyze = "European electric vehicles market" # Changed example to English
# ---------------------------------------------------------------------

print(f"Sector/Market to Analyze Set: '{sector_to_analyze}'")

Sector/Market to Analyze Set: 'European electric vehicles market'


In [4]:
# @title 4. Select LLM and Initialize Tools

# --- CHOOSE THE LANGUAGE MODEL (LLM) ---
try:
    # Try to initialize the OpenAI client. If it fails here, the API key is missing or invalid.
    llm = ChatOpenAI(model="gpt-4-turbo") # Powerful, but more expensive
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # Cheaper alternative
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify that the OpenAI API key is valid and loaded correctly (see Cell 2 output).")
    raise # Stop execution if the LLM cannot be created

# --- INITIALIZE TOOLS ---
# Web search tool (uses Serper.dev - requires SERPER_API_KEY)
search_tool = None
try:
    search_tool = SerperDevTool()
    print("SerperDevTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing SerperDevTool: {e}")
    print("--- Web search via Serper will not work. Ensure SERPER_API_KEY is set.")

# Tool to read/analyze website content
# WebsiteSearchTool is generally more robust for simple content.
web_reader_tool = WebsiteSearchTool()
print("WebsiteSearchTool initialized.")
# web_scraper_tool = ScrapeWebsiteTool() # Alternative for more complex sites, but more fragile
# print("ScrapeWebsiteTool initialized (alternative).")

# List of tools actually available for agents
available_tools = [tool for tool in [search_tool, web_reader_tool] if tool is not None]
print(f"Tools actually available for agents: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
WebsiteSearchTool initialized.
Tools actually available for agents: ['Search the internet with Serper', 'Search in a specific website']


In [5]:
# @title 5. Define New Agents (Market Analysis)

print("Defining New Agents for Market Analysis...")

# Verify that the LLM was initialized before creating agents
if 'llm' not in locals() or llm is None:
     raise ValueError("LLM was not initialized correctly in the previous cell. Cannot define agents.")

# --- Agent 1: Market Player Identifier ---
market_player_identifier = Agent(
    role='Market Research Analyst specializing in Sector Player Identification',
    goal=f'Identify the top 5 key players (companies) operating in the specified sector: "{sector_to_analyze}".',
    backstory=(
        "You are an expert market analyst skilled in using search engines and industry reports "
        "to identify the most influential companies, market leaders, or key innovators. "
        "Your goal is to map the competitive landscape by providing an accurate list of the main players."
    ),
    tools=[search_tool] if search_tool else [], # Use search_tool only if available
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5 # Limit iterations for safety
)
print("- Agent 'market_player_identifier' defined.")

# --- Agent 2: Company Info Gatherer ---
company_info_gatherer = Agent(
    role='Corporate Intelligence Data Collector',
    goal=f'For each of the key players identified, gather essential public information: a brief company profile, main products/services offered specifically for the "{sector_to_analyze}" sector, and (if available online) recent significant news or stated strategies related to this sector.',
    backstory=(
        "You are a meticulous researcher, a 'digital detective'. You know how to use web tools to extract "
        "relevant information from company websites, investor relations sections, press releases, and reliable news sources. "
        "You filter information to focus on what's pertinent to the specified sector and summarize the key points."
    ),
    tools=available_tools, # Provides available tools (search and web reading)
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=10 # More iterations might be needed to visit multiple sites/sources
)
print("- Agent 'company_info_gatherer' defined.")

# --- Agent 3: Sector Analyst ---
sector_analyst = Agent(
    role='Industry Trend Analyst',
    goal=f'Analyze the aggregated information collected on the key players to identify major trends (e.g., technological, business model, M&A), common business patterns, emerging challenges, or distinctive technologies prevalent within the "{sector_to_analyze}" sector.',
    backstory=(
        "You are a strategic analyst capable of seeing the 'big picture'. "
        "You connect fragmented information about different companies to outline the overall market dynamics. "
        "Your purpose is to identify driving forces, market segmentation, and potential future developments based on the collected data."
    ),
    # This agent works on provided data, no external tools needed
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'sector_analyst' defined.")

# --- Agent 4: Market Report Writer ---
market_report_writer = Agent(
    role='Market Analysis Report Writer',
    goal=f'Write a clear, concise, and well-structured market analysis report for the "{sector_to_analyze}" sector. The report should include: an introduction to the sector, brief profiles of the key players identified, and a summary of the main market trends/dynamics analyzed.',
    backstory=(
        "You are an expert communicator specializing in crafting accessible yet informative market analysis reports. "
        "You organize complex data into a coherent and logical narrative using professional language. "
        "Your goal is to produce a final report that highlights the key takeaways for a business reader."
    ),
    # This agent assembles and formats, no external tools needed
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'market_report_writer' defined.")

print("All new agents have been defined.")

Defining New Agents for Market Analysis...
- Agent 'market_player_identifier' defined.
- Agent 'company_info_gatherer' defined.
- Agent 'sector_analyst' defined.
- Agent 'market_report_writer' defined.
All new agents have been defined.


In [6]:
# @title 6. Define New Tasks (Market Analysis)

print("Defining New Tasks for Market Analysis...")

# Verify that agents were defined correctly
if 'market_player_identifier' not in locals() or \
   'company_info_gatherer' not in locals() or \
   'sector_analyst' not in locals() or \
   'market_report_writer' not in locals():
    raise ValueError("One or more required agents were not defined correctly.")

# --- Task 1: Identify Players ---
# Note: Using the agent variable name directly in f-string descriptions for context is generally
# less robust than relying on the 'context' parameter and letting CrewAI handle interpolation.
# However, for simple cases like passing the sector name, it's acceptable.
# For passing task outputs, using context=[] and letting CrewAI inject is preferred.
task_identify_players = Task(
    description=(
        f"Using the available web search tools, identify the top 5 main players (leading or most relevant companies) "
        f"operating specifically in the sector: '{sector_to_analyze}'. "
        f"Provide a concise numbered list containing only the official company name and the URL of their main website."
    ),
    agent=market_player_identifier,
    expected_output=(
        "A clear numbered list (1., 2., 3., 4., 5.) with company name and full URL for the top 5 identified players."
        " Example: '1. Alpha Company - https://www.alpha.com'"
    )
)
print("- Task 'task_identify_players' defined.")

# --- Task 2: Gather Company Data ---
task_gather_company_data = Task(
    description=(
        # CrewAI automatically injects the output of tasks defined in 'context'
        # So, we don't strictly need the placeholder like {task_identify_players.output} here,
        # but we describe that the list will be provided dynamically.
        f"For each of the 5 players identified in the previous task (the list will be provided dynamically in the context), "
        f"use the web tools to visit their sites and other reliable public sources (like news/investor sections or targeted searches). "
        f"Extract and concisely summarize the following information for EACH company, focusing on their activity in the '{sector_to_analyze}' sector:\n"
        f"1.  **Brief Company Profile:** 1-2 sentences about who they are and what they primarily do in this sector.\n"
        f"2.  **Key Products/Services in Sector:** List 2-4 of their main products or services specific to '{sector_to_analyze}'.\n"
        f"3.  **Recent News/Strategy (Optional, if found):** Mention 1-2 recent highlights (launches, acquisitions, partnerships, strategic statements) related to this sector.\n"
        f"Organize the output using Markdown, with an H3 title (### Company Name) for each of the 5 companies, followed by bullet points or short paragraphs for the requested information."
    ),
    agent=company_info_gatherer,
    expected_output=(
        "A Markdown formatted text containing 5 main sections (one per company, introduced by '### Company Name'). "
        "Each section must include: a brief profile, a list of key products/services in the sector, and optionally recent news/strategy."
    ),
    context=[task_identify_players], # Depends on the identification of players
    tools=available_tools # Make explicit which tools can be used for this task
)
print("- Task 'task_gather_company_data' defined.")

# --- Task 3: Analyze Sector ---
task_analyze_sector = Task(
    description=(
        f"Analyze the aggregated information gathered on the main players (provided in the context from the previous task) "
        f"related to the '{sector_to_analyze}' sector. "
        f"Synthesize and describe in a brief paragraph or bulleted list (3-5 points) the main findings regarding:\n"
        f"-   **Common Trends:** What approaches, technologies, or business models seem prevalent or emerging among these players?\n"
        f"-   **Segmentation or Niches:** Are there clear differences in how these players position themselves or the sub-segments they target?\n"
        f"-   **Potential Challenges:** Are common challenges mentioned or inferable (e.g., regulation, competition, technology)?\n"
        f"Base the analysis solely on the data provided in the context."
    ),
    agent=sector_analyst,
    expected_output=(
        "A concise analysis (paragraph or bullet points) highlighting 3-5 of the main observed trends, business models, segmentations, or challenges in the sector, based on the aggregated data of the main players."
    ),
    context=[task_gather_company_data] # Depends on the gathered company data
)
print("- Task 'task_analyze_sector' defined.")

# --- Task 4: Write Market Report ---
task_write_market_report = Task(
    description=(
        # References to previous tasks outputs are handled by CrewAI via the 'context' list.
        # We describe the structure and rely on the agent using the context provided.
        f"Write a concise and professional final market analysis report (approx. 500-700 words) for the sector: '{sector_to_analyze}'. "
        f"The report must have the following structure:\n"
        f"1.  **Introduction (Brief):** Briefly describe the '{sector_to_analyze}' sector and the report's purpose.\n"
        f"2.  **Key Market Players:** Present brief profiles for each of the 5 players identified (use information from the company data gathering task, synthesizing further if needed. Include player names from the identification task).\n"
        f"3.  **Key Sector Trends and Dynamics:** Report the summary of the market trends and dynamics analysis from the previous task.\n"
        f"4.  **Conclusion (Optional, Brief):** A final summary sentence.\n"
        f"Use clear and professional language. Ensure the context from previous tasks is correctly utilized."
    ),
    agent=market_report_writer,
    expected_output=(
        "A well-formatted final report (plain text or Markdown), approx. 500-700 words, "
        "structured as requested (Introduction, Key Players, Trends/Dynamics, Optional Conclusion), "
        "effectively summarizing all information gathered and analyzed in previous tasks."
    ),
    # Depends on all previous tasks for names, company data, and trend analysis
    context=[task_identify_players, task_gather_company_data, task_analyze_sector]
)
print("- Task 'task_write_market_report' defined.")

print("All new tasks have been defined.")

Defining New Tasks for Market Analysis...
- Task 'task_identify_players' defined.
- Task 'task_gather_company_data' defined.
- Task 'task_analyze_sector' defined.
- Task 'task_write_market_report' defined.
All new tasks have been defined.


In [7]:
# @title 7. Create and Run the New Crew (Market Analysis)

print("Creating the New Crew (Market Analysis)...")

# Verify that all new components are ready
if 'market_player_identifier' not in locals() or \
   'company_info_gatherer' not in locals() or \
   'sector_analyst' not in locals() or \
   'market_report_writer' not in locals() or \
   'task_identify_players' not in locals() or \
   'task_gather_company_data' not in locals() or \
   'task_analyze_sector' not in locals() or \
   'task_write_market_report' not in locals():
    raise ValueError("Missing agents or tasks to create the Market Analysis Crew.")

# Assemble the New Crew with the new agents and tasks
market_analysis_crew = Crew(
    agents=[market_player_identifier, company_info_gatherer, sector_analyst, market_report_writer],
    tasks=[task_identify_players, task_gather_company_data, task_analyze_sector, task_write_market_report],
    process=Process.sequential, # Tasks will run in sequential order
    memory=True, # Enable shared memory between tasks to maintain context
    cache=True, # Enable caching for tools (saves API/web calls for identical inputs)
    verbose=True # Use True for maximum detail, False for less output
)

print("New Crew created. Starting the process with kickoff()...")

# Define the initial inputs for the crew
# Passing the sector here provides initial context, though it's also in the f-string prompts.
crew_inputs = {
    'sector': sector_to_analyze
}

# Execute the entire sequence of tasks defined in the Crew
# This might take time and consume API credits!
result = market_analysis_crew.kickoff(inputs=crew_inputs) # <-- Use kickoff() here!

print("\n\n*************************************")
print("   NEW CREW HAS FINISHED WORK!       ")
print("*************************************")

Creating the New Crew (Market Analysis)...
New Crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 094bf7f6-bb3c-4a96-bf0b-bf88ef4e3288                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 72e5fa02-1c0b-466e-b348-74886372ba1f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 72e5fa02-1c0b-466e-b348-74886372ba1f
       Status: Executing Task...
    └── 🤖 Agent: Market Research Analyst specializing in Sector Player Identification
            Status: In Progress

# Agent: Market Research Analyst specializing in Sector Player Identification
## Task: Using the available web search tools, identify the top 5 main players (leading or most relevant companies) operating specifically in the sector: 'European electric vehicles market'. Provide a concise numbered list containing only the official company name and the URL of their main website.


...

🚀 Crew: crew
├── 📋 Task: 72e5fa02-1c0b-466e-b348-74886372ba1f
│      Assigned to: Market Research Analyst specializing in Sector Player Identification
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Research Analyst specializing in Sector Player Identification
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 27111806-5d42-47cc-a2b8-91acabe08f89
│      Assigned to: Corporate Intelligence Data Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: Corporate Intelligence Data Collector
│           Status: ✅ Completed
│       ├── 🔧 Used Search in a specific website (5)
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: d4c14270-e089-44d9-bbe3-e1a3a49d9a46
│      Assigned to: Industry Trend Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Industry Trend Analyst
│           Status: ✅ Completed
└── 📋 Task: 43659a51-f5e0-4ba0-906a-577628d80931
       Status: Executing Task...
    └── 🤖 Agent: Market Analysis Report Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 72e5fa02-1c0b-466e-b348-74886372ba1f
│      Assigned to: Market Research Analyst specializing in Sector Player Identification
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Research Analyst specializing in Sector Player Identification
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 27111806-5d42-47cc-a2b8-91acabe08f89
│      Assigned to: Corporate Intelligence Data Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: Corporate Intelligence Data Collector
│           Status: ✅ Completed
│       ├── 🔧 Used Search in a specific website (5)
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: d4c14270-e089-44d9-bbe3-e1a3a49d9a46
│      Assigned to: Industry Trend Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Industry Trend Analyst
│           Status: ✅ Completed
└── 📋 Task: 43659a51-f5e0-4ba0-906a-577628d80931
       Assigned to: Market Analysis Report Writer
       Status: ✅ Completed
    └── 🤖 Agent: Market Analysis Report Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 43659a51-f5e0-4ba0-906a-577628d80931                                                                     │
│  Agent: Market Analysis Report Writer                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 094bf7f6-bb3c-4a96-bf0b-bf88ef4e3288                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*************************************
   NEW CREW HAS FINISHED WORK!       
*************************************


In [8]:
# @title 8. Display the Final Result (Market Report)

print("\n\n#####################################")
print("## Final Market Analysis Report:")
print("#####################################\n")

# Print the final result produced by the last task (task_write_market_report)
if 'result' in locals():
    # Print the result, which should be the final report
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")



#####################################
## Final Market Analysis Report:
#####################################

### European Electric Vehicles Market Analysis Report

#### Introduction
The European electric vehicles (EV) market is experiencing rapid growth and transformation, fueled by stringent environmental regulations, technological advancements, and shifting consumer preferences towards sustainable transportation. This report delves into the dynamics of the market, highlighting key players, trends, and strategic shifts shaping the future of electric mobility in Europe.

#### Key Market Players

**Tesla, Inc.**
An American powerhouse in electric vehicles, Tesla, Inc. is known for its innovative approach to EVs and clean energy solutions. Key offerings include the Tesla Model 3 and Model Y. Despite its strong brand, Tesla has seen a recent decline in its European market share, facing fierce competition from local manufacturers.

**Volkswagen AG**
Volkswagen AG stands as a major Germa